### Download HRRR data from U of UTAH CHPC archive
Wei-Ming Tsai, Nov. 2020 <br>
HRRR data archive: http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/hrrr_script_tips.html

In [3]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
#import cfgrib
import cf2cdm
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import xarray as xr
from datetime import datetime, timedelta
import urllib.request
#from cfgrib.xarray_store import open_dataset
import warnings
import h5py

/home/willytsai/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
warnings.filterwarnings('ignore')

In [ ]:
# var = 'PWAT'
# id_begin = 49173818
# id_end = 50061135

In [35]:
file.time[9]

<xarray.DataArray 'time' ()>
array('2014-02-28T21:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2014-02-28T21:00:00
Attributes:
    standard_name:  time
    long_name:      time
    axis:           T

In [36]:
tmp = datetime.utcfromtimestamp(file.time[9].values.tolist()/1e9)
time_str = tmp.strftime('%Y%m%d')
time_str

'20140228'

In [37]:
os.chdir('/w2-data2/willytsai/InSAR_HRRR/MERRA2_ARs/')
file = xr.open_dataset('date_ARs_events.nc')
date_acqui = []
for t in range(len(file.time)):
    tmp = datetime.utcfromtimestamp(file.time[t].values.tolist()/1e9)
    time_str = tmp.strftime('%Y%m%d')
    date_acqui.append(time_str) # times of acquisitions

In [39]:
len(date_acqui)

120

### Download HRRR data from archive

In [ ]:
os.chdir('/data2/willytsai/InSAR_HRRR/HRRR_data/t02z/prs_levels/')
# Downloading best analyzed output for specfied UTC.
# example: 
for date in date_acqui:
    for hour in ['02']:
        file_name = 'hrrr.'+date+'.t'+hour+'z.grib2'
        url = 'curl -o ' + file_name + \
          ' https://pando-rgw01.chpc.utah.edu/hrrr/prs/'+date+'/hrrr.t'+hour+'z.wrfprsf00.grib2'
        os.system(url)

### Extract precipitable water and merge into a single ncfile

In [ ]:
# get geolocation from InSAR
geo_file = '/data2/willytsai/InSAR_HRRR/CentralNevadaSenAT166/mintpy/inputs/geometryRadar.h5'
geo = h5py.File(geo_file,'r')
# for key in geo.keys():
#     print(key) #Names of the groups in HDF5 file.
lat = geo['latitude'];
lon = geo['longitude'];
incidence = geo['incidenceAngle'];
axis_bound = [np.min(lat),np.max(lat),np.min(lon),np.max(lon)]; # coordinate bound [South,North,West,East]
axis_bound = [np.unique(lat.value)[1],np.unique(lat.value)[-1],np.unique(lon.value)[0],np.unique(lon.value)[-2]]
axis_bound

In [ ]:
# merge data

# 0. get the geolocation and array size of HRRR
os.chdir('/data2/willytsai/InSAR_HRRR/HRRR_data/t02z')
files = sorted(glob('*grib2'))
ds = xr.open_dataset('hrrr.20200313.t02z.grib2',engine='cfgrib',
                    backend_kwargs=dict(filter_by_keys={'typeOfLevel':'unknown'}))

lon = ds.longitude-360 # convert to -180-180
lat = ds.latitude

lat_idx = np.where(np.logical_and(lat>=axis_bound[0],lat<=axis_bound[1]))
lon_idx = np.where(np.logical_and(lon>=axis_bound[2],lon<=axis_bound[3]))
# get x and y index for the targed region
idy_list = np.intersect1d(lat_idx[0],lon_idx[0])
idx_list = np.intersect1d(lat_idx[1],lon_idx[1])

In [ ]:
os.chdir('/data2/willytsai/InSAR_HRRR/HRRR_data/t02z')
pwat_acqui = np.zeros((len(files),len(idy_list),len(idx_list)))
date_frame = []

for t in range(len(files)):
    
    date_frame.append(datetime.strptime(files[t][5:18],'%Y%m%d.t%Hz'))
    try:
        ds = xr.open_dataset(files[t],engine='cfgrib',
                            backend_kwargs=dict(filter_by_keys={'typeOfLevel':'unknown'}))
        pwat = ds.pwat
        pwat_acqui[t,:,:] = pwat[idy_list[0]:idy_list[-1]+1,idx_list[0]:idx_list[-1]+1]    
    except:
        print('ERROR file: '+files[t])
        pwat_acqui[t,:,:] = np.nan

# create a merged directory for merged file of PWAT over NEVADA
os.system('mkdir -p merged')
np.save('./merged/HRRR_lon_acquisition.npy',lon[idy_list[0]:idy_list[-1]+1,idx_list[0]:idx_list[-1]+1].values)
np.save('./merged/HRRR_lat_acquisition.npy',lat[idy_list[0]:idy_list[-1]+1,idx_list[0]:idx_list[-1]+1].values)
np.save('./merged/HRRR_time_acquisition.npy',date_frame)
np.save('./merged/HRRR_PWAT_acquisition.npy',pwat_acqui)

### Checking merged files

In [ ]:
lon1 = np.load('./merged/HRRR_lon_acquisition.npy')
lat1 = np.load('./merged/HRRR_lat_acquisition.npy')
date1 = np.load('./merged/HRRR_time_acquisition.npy',allow_pickle=True)
pwat1 = np.load('./merged/HRRR_PWAT_acquisition.npy')

In [ ]:
date1[:30]

In [ ]:
row

In [ ]:
total_frame = len(date1)
rows,remainder = np.divmod(161,21)
col = np.repeat(np.arange(0,rows+1),21)
row = np.tile(np.arange(0,21),rows+1)

fig,ax = plt.subplots(21,rows+1,figsize=(20,20))
#ax = plt.subplot(projection=ccrs.PlateCarree())
for t in range(total_frame):
    ax[row[t],col[t]].pcolor(lon1,lat1,pwat1[t,:,:],transform=ccrs.PlateCarree(),vmin=0,vmax=30,cmap='jet')
    ax[row[t],col[t]].set_ylim([37.228, 43.223568])
    ax[row[t],col[t]].set_xlim([-119.00549, -114.65358])
    ax[row[t],col[t]].set_title(date1[t])
#    ax.coastlines()
#ax.gridlines(draw_labels=False)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = plt.subplot(projection=ccrs.PlateCarree())
plt.pcolor(lon1,lat1,pwat1[0,:,:],transform=ccrs.PlateCarree(),vmin=0,vmax=30,cmap='jet')
#plt.ylim([37.228, 43.223568])
plt.xlim([-119.00549, -114.65358])
ax.coastlines()
#ax.gridlines(draw_labels=False)